## Import necessary libraries

In [48]:
import pandas as pd
import numpy as np


## Load data

In [49]:
df = pd.read_csv('/kaggle/input/all-posts/all_posts.csv', nrows=1000000)
df.head()

,postID,originalContent,date,sentiment,taggedSymbols
0,3118126,Vào NKG ngày mai ổn không các bác? Dài hạn vào...,2021-09-30,0.0,['NKG']
1,3118104,Có một điều thấy buồn cười nhất là những người...,2021-09-30,0.0,['ART']
2,3118086,Lại là ĐTC,2021-09-30,0.0,['DHA']
3,3118058,"Chào anh em, mình gửi anh chị em chiến lược đá...",2021-09-30,0.0,[]
4,3118053,CTC vào 8.3 có dính bô không mọi người?,2021-09-30,0.0,['CTC']


## Keep posts with negative and positive sentiment only

In [50]:
sentiment_df = df[df['sentiment'].isin([-1, 1])]
sentiment_df.head()

,postID,originalContent,date,sentiment,taggedSymbols
11,3117847,NÊN MUA GÌ CHO BÁO CÁO QUÝ 3?\n\nTháng 10 là g...,2021-09-30,1.0,"['DCM', 'DPM', 'GMD', 'HPG', 'KSB', 'NTL', 'PV..."
29,3117591,Dự án Akari City với quy mô 5.000 căn hộ nằm t...,2021-09-30,1.0,['NLG']
31,3117571,"PNJ - HÀNH TRÌNH MỚI 🔥🔥\n\nPNJ + 5,79% 🔥\n\nTạ...",2021-09-30,1.0,"['MSN', 'MWG', 'PC1', 'PNJ']"
48,3117327,Tiếc quá không CE\nNay ăn được PNJ em vui quá ...,2021-09-30,1.0,['PNJ']
56,3117230,Giá khí đốt lại tăng dựng đứng mai lại trần ti...,2021-09-30,1.0,['ASP']


## Data Cleaning

In [51]:
import re

def remove_links(text):
    # Xóa link dạng http(s)://... hoặc www....
    text = re.sub(r"http\S+", "", text)     # remove http:// hoặc https://
    text = re.sub(r"www\.\S+", "", text)    # remove www...
    text = re.sub(r"\S+\.com\S*", "", text) # remove .com/.net/.vn...
    return text.strip()

def clean_text(text):
    text = str(text).lower()                          # lowercase
    text = re.sub(r"\n+", ". ", text)                # replace new line with period
    text = remove_links(text)                         # remove links
    text = re.sub(r"@\w+", "", text)                  # remove mentions (@abc)
    text = re.sub(r"#\w+", "", text)                  # remove hashtags
    text = re.sub(
        r"[^0-9a-zA-Záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệ"
        r"íìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữự"
        r"ýỳỷỹỵđ\s!?,.]+", 
        " ", 
        text
    )            # keep only letters, numbers and some punctuation
    text = re.sub(r"\s+", " ", text).strip()          # remove extra spaces
    return text


### Remove links in posts content

In [52]:
sentiment_df['originalContent'] = sentiment_df['originalContent'].apply(clean_text)
sentiment_df.head()


/tmp/ipykernel_191/952562065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df['originalContent'] = sentiment_df['originalContent'].apply(clean_text)


,postID,originalContent,date,sentiment,taggedSymbols
11,3117847,nên mua gì cho báo cáo quý 3?. tháng 10 là gia...,2021-09-30,1.0,"['DCM', 'DPM', 'GMD', 'HPG', 'KSB', 'NTL', 'PV..."
29,3117591,dự án akari city với quy mô 5.000 căn hộ nằm t...,2021-09-30,1.0,['NLG']
31,3117571,"pnj hành trình mới . pnj 5,79 . tại sao nói hà...",2021-09-30,1.0,"['MSN', 'MWG', 'PC1', 'PNJ']"
48,3117327,tiếc quá không ce. nay ăn được pnj em vui quá ...,2021-09-30,1.0,['PNJ']
56,3117230,giá khí đốt lại tăng dựng đứng mai lại trần ti...,2021-09-30,1.0,['ASP']


## Data Preparation

In [53]:
from transformers import AutoTokenizer 
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


In [54]:
import torch
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=100, mode="train",test_size=0.2, random_state=42):
        # keep only necessary columns
        df = df[['originalContent', 'sentiment']]
        
        # train/test split
        train_df, test_df = train_test_split(
            df, 
            test_size=test_size,
            stratify=df['sentiment'],
            random_state=random_state
        )
        
        self.df = train_df if mode == "train" else test_df 
        
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row['originalContent']
        label = 1 if row['sentiment'] == 1 else 0
        
        tokens = self.tokenizer(
            text, 
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        return {
            "input_ids": tokens['input_ids'].squeeze(0),
            "attention_mask": tokens['attention_mask'].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
            }

In [55]:
from torch.utils.data import WeightedRandomSampler
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("5CD-AI/Vietnamese-Sentiment-visobert")

# Create dataset
train_dataset = SentimentDataset(sentiment_df, tokenizer, mode='train')
test_dataset = SentimentDataset(sentiment_df, tokenizer, mode='test')

# compute class weights
labels = train_dataset.df['sentiment'].map({-1:0, 1:1}).values
class_counts = [sum(labels==0), sum(labels==1)] # [neg_count, pos_count]
class_weights = [1.0 / count for count in class_counts]

# assign weight to each sample
sample_weights = [class_weights[label] for label in labels]

# create WeightedRandomSampler
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True # allow sampling the same example multiple times
)

# create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# check a batch
batch = next(iter(train_loader))
print(batch['input_ids'].shape)  # (batch_size, seq_len)
print(batch['labels'])

torch.Size([32, 100])
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1])


In [56]:
# check a batch
batch = next(iter(train_loader))
print(batch['input_ids'].shape)  # (batch_size, seq_len)
print(batch['labels'])
print(batch['labels'].bincount()) 

torch.Size([32, 100])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
        0, 1, 1, 1, 0, 0, 1, 1])
tensor([13, 19])


## Load pretrained model

In [ ]:
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoConfig
model_name = "5CD-AI/Vietnamese-Sentiment-visobert"

tokenizer = AutoTokenizer.from_pretrained(model_name)


# 1. Load config
config = AutoConfig.from_pretrained("5CD-AI/Vietnamese-Sentiment-visobert")
config.num_labels = 2  # important

# 2. Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True   # avoid shape errors
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at 5CD-AI/Vietnamese-Sentiment-visobert and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
def freeze_model_layers(model, unfreeze_last_n=4):
    """
    Freeze all but the last `n` transformer layers of the encoder,
    plus keep the classifier trainable.
    """
    # 1. Freeze embeddings
    for param in model.roberta.embeddings.parameters():
        param.requires_grad = False

    # 2. Freeze all encoder layers except the last `unfreeze_last_n`
    for i, layer in enumerate(model.roberta.encoder.layer):
        if i < len(model.roberta.encoder.layer) - unfreeze_last_n:
            for param in layer.parameters():
                param.requires_grad = False
        else:
            for param in layer.parameters():
                param.requires_grad = True

    # 3. Always train classifier head
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Print summary
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable_params/1e6:.2f}M / {total_params/1e6:.2f}M total "
          f"({100 * trainable_params/total_params:.1f}%)")

    return model


In [59]:
freeze_model_layers(model)

Trainable params: 28.94M / 97.57M total (29.7%)


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(15004, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=7

In [60]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Evaluation function
def evaluate_model(model, data_loader, device='cuda'):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            with autocast('cuda'):  # Add this for mixed precision
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits 
            
            batch_preds = torch.argmax(logits, dim=1)
            preds.extend(batch_preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            
    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)
    precision = precision_score(true_labels, preds)
    recall = recall_score(true_labels, preds)

    print(f"Eval | Acc: {acc:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
    return acc, f1, precision, recall

def train_model(model, train_loader, val_loader=None,
                epochs=3, lr=2e-5, weight_decay=0.01,
                warmup_ratio=0.1, max_grad_norm=1.0, device='cuda',
                save_dir="./checkpoints", save_every_n_epochs=2):
    
    os.makedirs(save_dir, exist_ok=True)
    model.to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    total_steps = len(train_loader) * epochs
    warmup_steps = int(total_steps * warmup_ratio)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    scaler = GradScaler()

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            
            with autocast(device_type='cuda'):
                outputs = model(input_ids=input_ids,
                                attention_mask=attention_mask,
                                labels=labels)
                loss = outputs.loss
                
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Train loss: {avg_loss:.4f}")
        
        # Evaluate
        if val_loader is not None:
            evaluate_model(model, val_loader, device=device)
        
        # === SAVE EVERY `save_every_n_epochs` EPOCHS ===
        if (epoch + 1) % save_every_n_epochs == 0:
            save_path = os.path.join(save_dir, f"model_epoch_{epoch+1}")
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
            print(f"Checkpoint saved: {save_path}")

    # Save final model
    final_path = os.path.join(save_dir, "model_final")
    model.save_pretrained(final_path)
    tokenizer.save_pretrained(final_path)
    print(f"Final model saved: {final_path}")
    
    print("Training complete!")

In [61]:
train_model(model, train_loader, val_loader=test_loader, epochs=10)

Training Epoch 1: 100%|██████████| 3347/3347 [05:05<00:00, 10.97it/s]


Epoch 1 | Train loss: 0.5141


Eval | Acc: 0.7735 | F1: 0.8329 | Precision: 0.9295 | Recall: 0.7545


Training Epoch 2: 100%|██████████| 3347/3347 [05:05<00:00, 10.97it/s]


Epoch 2 | Train loss: 0.4010


...

Eval | Acc: 0.8413 | F1: 0.8916 | Precision: 0.9114 | Recall: 0.8727


Training Epoch 10: 100%|██████████| 3347/3347 [05:04<00:00, 11.00it/s]


Epoch 10 | Train loss: 0.1415


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:407: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 256}
  warnings.warn(


Eval | Acc: 0.8395 | F1: 0.8900 | Precision: 0.9132 | Recall: 0.8680
Checkpoint saved: ./checkpoints/model_epoch_10
Final model saved: ./checkpoints/model_final
Training complete!


## Apply the Model for Neutral Sentiment Posts

In [62]:
import torch
from torch.amp import autocast
from tqdm import tqdm

@torch.no_grad()
def analyze_zero_sentiment_posts(
    df,
    model,
    tokenizer,
    text_column='originalContent',
    sentiment_column='sentiment',
    batch_size=64,
    device='cuda',
    label_map={0: -1, 1: 1}  # 0 in logits → negative, 1 → positive
):
    """
    Predict sentiment for rows where `sentiment == 0` (unknown/neutral) efficiently
    for large datasets.

    Args:
        df (pd.DataFrame): Input dataframe
        model: Trained HuggingFace model
        tokenizer: Corresponding tokenizer
        text_column: Name of column with text
        sentiment_column: Name of column with labels (0, 1, -1)
        batch_size: Inference batch size
        device: 'cuda' or 'cpu'
        label_map: Map from model output logits → sentiment label

    Returns:
        df_with_pred: DataFrame with new column `predicted_sentiment`
    """

    # Filter rows with sentiment == 0
    zero_df = df[df[sentiment_column] == 0].copy()
    if zero_df.empty:
        print("No rows with sentiment == 0 found.")
        df['predicted_sentiment'] = None
        return df

    print(f"Analyzing {len(zero_df)} posts with sentiment == 0...")

    model.eval()
    model.to(device)

    # Clean text column efficiently
    zero_df[text_column] = [clean_text(str(t)) for t in zero_df[text_column].fillna("")]

    pred_labels = []

    # Batch inference
    for start_idx in tqdm(range(0, len(zero_df), batch_size), desc="Predicting"):
        batch_texts = zero_df[text_column].iloc[start_idx:start_idx+batch_size].tolist()
        
        encodings = tokenizer(
            batch_texts,
            truncation=True,
            padding=True,
            max_length=256,
            return_tensors='pt'
        )

        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)

        with autocast(device_type='cuda' if 'cuda' in device else 'cpu'):
            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            preds = torch.argmax(logits, dim=-1)

        pred_labels.extend([label_map[int(p)] for p in preds.cpu()])

        # Free GPU memory after each batch
        del input_ids, attention_mask, logits, preds
        torch.cuda.empty_cache()

    # Assign predictions directly to original df
    df_with_pred = df.copy()
    df_with_pred['predicted_sentiment'] = None
    df_with_pred.loc[df[sentiment_column] == 0, 'predicted_sentiment'] = pred_labels

    # Summary
    pos = sum(p == 1 for p in pred_labels)
    neg = sum(p == -1 for p in pred_labels)
    print(f"\nResults for sentiment == 0 posts:")
    print(f"  Positive: {pos} ({pos/len(pred_labels)*100:.1f}%)")
    print(f"  Negative: {neg} ({neg/len(pred_labels)*100:.1f}%)")

    return df_with_pred


In [63]:
new_df = pd.read_csv('/kaggle/input/all-posts/all_posts.csv')
new_df.head()

,postID,originalContent,date,sentiment,taggedSymbols
0,3118126,Vào NKG ngày mai ổn không các bác? Dài hạn vào...,2021-09-30,0.0,['NKG']
1,3118104,Có một điều thấy buồn cười nhất là những người...,2021-09-30,0.0,['ART']
2,3118086,Lại là ĐTC,2021-09-30,0.0,['DHA']
3,3118058,"Chào anh em, mình gửi anh chị em chiến lược đá...",2021-09-30,0.0,[]
4,3118053,CTC vào 8.3 có dính bô không mọi người?,2021-09-30,0.0,['CTC']


In [65]:
new_df = analyze_zero_sentiment_posts(
    df=new_df,           # original dataframe
    model=model,               # trained model
    tokenizer=tokenizer,       # same tokenizer
    text_column='originalContent',
    sentiment_column='sentiment',
    batch_size=32,
    device='cuda'
)

new_df.head()

Analyzing 5709889 posts with sentiment == 0...


Predicting: 100%|██████████| 178435/178435 [2:40:54<00:00, 18.48it/s]  



Results for sentiment == 0 posts:
  Positive: 3394107 (59.4%)
  Negative: 2315782 (40.6%)


,postID,originalContent,date,sentiment,taggedSymbols,predicted_sentiment
0,3118126,Vào NKG ngày mai ổn không các bác? Dài hạn vào...,2021-09-30,0.0,['NKG'],1
1,3118104,Có một điều thấy buồn cười nhất là những người...,2021-09-30,0.0,['ART'],-1
2,3118086,Lại là ĐTC,2021-09-30,0.0,['DHA'],1
3,3118058,"Chào anh em, mình gửi anh chị em chiến lược đá...",2021-09-30,0.0,[],1
4,3118053,CTC vào 8.3 có dính bô không mọi người?,2021-09-30,0.0,['CTC'],1


In [66]:
output_path = '/kaggle/working/posts_sentiment.csv'
new_df.to_csv(output_path, index=False)


In [69]:
from IPython.display import FileLink

FileLink("/kaggle/working/posts_sentiment.csv")

/kaggle/working/posts_sentiment.csv

In [70]:
import shutil

# Zip the file
shutil.make_archive("/kaggle/working/posts_sentiment", "zip", root_dir="/kaggle/working", base_dir="posts_sentiment.csv")


'/kaggle/working/posts_sentiment.zip'

In [71]:
import shutil

# Zip the final model folder
shutil.make_archive("/kaggle/working/final_model", "zip", "/kaggle/working/checkpoints/model_final")

# Now Kaggle UI will show final_model.zip in the Output panel for download


'/kaggle/working/final_model.zip'

## Group Sentiment Count by Stock Symbols and Date

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fireant_data/sentiment_posts/posts_sentiment.csv')
df

,postID,originalContent,date,sentiment,taggedSymbols,predicted_sentiment
0,3118126,Vào NKG ngày mai ổn không các bác? Dài hạn vào...,2021-09-30,0.0,['NKG'],1.0
1,3118104,Có một điều thấy buồn cười nhất là những người...,2021-09-30,0.0,['ART'],-1.0
2,3118086,Lại là ĐTC,2021-09-30,0.0,['DHA'],1.0
3,3118058,"Chào anh em, mình gửi anh chị em chiến lược đá...",2021-09-30,0.0,[],1.0
4,3118053,CTC vào 8.3 có dính bô không mọi người?,2021-09-30,0.0,['CTC'],1.0
...,...,...,...,...,...,...
6558259,35723686,Giữa tháng 10 lên 27 quá,2025-10-01,0.0,['USD-VND'],1.0
6558260,35729297,Từ từ thôi còn sớm lắm.,2025-10-01,0.0,['SSI'],1.0
6558261,35729295,Thu già với thành thanh hoá thăng thiên sao mà...,2025-10-01,0.0,['SGR'],-1.0
6558262,35729294,T+ bát hết đi cho nhẹ tàu để anh em chúng tôi ...,2025-10-01,0.0,['EVF'],1.0


In [3]:
# Remove rows that do not mention any stock symbols
print("Original number of rows:", len(df))
df = df[df['taggedSymbols'] != '[]']
print("Number of rows after removing no-symbol posts:", len(df))



Original number of rows: 6558264
Number of rows after removing no-symbol posts: 6425289


In [9]:
# set sentiment = predicted_sentiment where sentiment == 0
df.loc[df['sentiment'] == 0, 'sentiment'] = df.loc[df['sentiment'] == 0, 'predicted_sentiment']
df

,postID,originalContent,date,sentiment,taggedSymbols,predicted_sentiment
0,3118126,Vào NKG ngày mai ổn không các bác? Dài hạn vào...,2021-09-30,1.0,['NKG'],1.0
1,3118104,Có một điều thấy buồn cười nhất là những người...,2021-09-30,-1.0,['ART'],-1.0
2,3118086,Lại là ĐTC,2021-09-30,1.0,['DHA'],1.0
4,3118053,CTC vào 8.3 có dính bô không mọi người?,2021-09-30,1.0,['CTC'],1.0
5,3117995,"VIB ae cứ canh 35,x mua đợi 1-2 tuần lên 38, 3...",2021-09-30,1.0,['VIB'],1.0
...,...,...,...,...,...,...
6558259,35723686,Giữa tháng 10 lên 27 quá,2025-10-01,1.0,['USD-VND'],1.0
6558260,35729297,Từ từ thôi còn sớm lắm.,2025-10-01,1.0,['SSI'],1.0
6558261,35729295,Thu già với thành thanh hoá thăng thiên sao mà...,2025-10-01,-1.0,['SGR'],-1.0
6558262,35729294,T+ bát hết đi cho nhẹ tàu để anh em chúng tôi ...,2025-10-01,1.0,['EVF'],1.0


In [4]:
# Drop originalContent and predicted_sentiment columns
df = df.drop(columns=['originalContent', 'predicted_sentiment'])

## Explode taggedSymbols

In [5]:
import ast

# Convert string to list
df['taggedSymbols'] = df['taggedSymbols'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

df

,postID,date,sentiment,taggedSymbols
0,3118126,2021-09-30,0.0,[NKG]
1,3118104,2021-09-30,0.0,[ART]
2,3118086,2021-09-30,0.0,[DHA]
4,3118053,2021-09-30,0.0,[CTC]
5,3117995,2021-09-30,0.0,[VIB]
...,...,...,...,...
6558259,35723686,2025-10-01,0.0,[USD-VND]
6558260,35729297,2025-10-01,0.0,[SSI]
6558261,35729295,2025-10-01,0.0,[SGR]
6558262,35729294,2025-10-01,0.0,[EVF]


In [6]:
df = df.explode('taggedSymbols').reset_index(drop=True)
df

,postID,date,sentiment,taggedSymbols
0,3118126,2021-09-30,0.0,NKG
1,3118104,2021-09-30,0.0,ART
2,3118086,2021-09-30,0.0,DHA
3,3118053,2021-09-30,0.0,CTC
4,3117995,2021-09-30,0.0,VIB
...,...,...,...,...
9497549,35723686,2025-10-01,0.0,USD-VND
9497550,35729297,2025-10-01,0.0,SSI
9497551,35729295,2025-10-01,0.0,SGR
9497552,35729294,2025-10-01,0.0,EVF


In [7]:
# Group Sentiment Count by Stock Symbols and Date
df = df.groupby(['date', 'taggedSymbols']).agg(
    num_positive=('sentiment', lambda x: (x == 1).sum()),
    num_negative=('sentiment', lambda x: (x == -1).sum()),
    num_neutral=('sentiment', lambda x: (x == 0).sum()),
    total_posts=('sentiment', 'count')
).reset_index()

df

,date,taggedSymbols,num_positive,num_negative,num_neutral,total_posts
0,2021-09-01,AAA,2,0,20,22
1,2021-09-01,AAS,0,0,3,3
2,2021-09-01,AAT,0,0,4,4
3,2021-09-01,AAV,0,0,5,5
4,2021-09-01,ABB,1,0,7,8
...,...,...,...,...,...,...
565000,2025-10-31,^DJI,3,2,36,41
565001,2025-10-31,^HSI,0,0,2,2
565002,2025-10-31,^IXIC,0,0,1,1
565003,2025-10-31,^KS11,0,0,4,4


In [8]:
df.to_csv('fireant_data/sentiment_posts/sentiment_posts_count.csv', index=False)